In [1]:
from google.colab import drive
drive.mount('/content/drive')





Mounted at /content/drive


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

# ✅ تحميل البيانات الأصلية
file_path = "/content/drive/My Drive/Colab Notebooks/train.csv"
df = pd.read_csv(file_path, low_memory=False)

# ✅ التأكد من وجود عمود Clade
if 'Clade' not in df.columns:
    raise ValueError("❌ العمود 'Clade' غير موجود في البيانات!")

# ✅ تخزين بيانات التدريب والتحقق
train_list = []
val_list = []

# ✅ تقسيم البيانات لكل Clade على حدة
for clade, group in df.groupby('Clade'):
    # 🔹 التأكد من وجود عدد كافٍ من العينات
    if len(group) < 50000:
        raise ValueError(f"⚠️ عدد العينات في Clade {clade} أقل من 50,000!")

    # 🔹 أخذ 50,000 عينة لبيانات التدريب
    train_clade = group.sample(n=50000, random_state=42)

    # 🔹 باقي البيانات تذهب إلى مجموعة التحقق (Validation)
    val_clade = group.drop(train_clade.index)

    # ✅ إضافة البيانات إلى القوائم
    train_list.append(train_clade)
    val_list.append(val_clade)

# ✅ تجميع البيانات المقسمة
df_train = pd.concat(train_list).reset_index(drop=True)
df_val = pd.concat(val_list).reset_index(drop=True)

# ✅ التأكد من الأحجام النهائية
if len(df_train) != 350000 or len(df_val) != 210000:
    raise ValueError("❌ حجم بيانات التدريب أو التحقق غير صحيح!")

# ✅ حفظ البيانات الجديدة
train_output = "/content/drive/My Drive/Colab Notebooks/train_split.csv"
val_output = "/content/drive/My Drive/Colab Notebooks/val_split.csv"

df_train.to_csv(train_output, index=False)
df_val.to_csv(val_output, index=False)

print(f"✅ تم تقسيم البيانات وحفظها بنجاح:")
print(f"🔹 بيانات التدريب: {train_output} ({len(df_train)} صف)")
print(f"🔹 بيانات التحقق: {val_output} ({len(df_val)} صف)")


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score, roc_auc_score
from sklearn.preprocessing import LabelEncoder
import pickle

# ✅ تحميل بيانات التدريب الكاملة
train_file = "/content/drive/My Drive/Colab Notebooks/train_split.csv"
df_train = pd.read_csv(train_file, low_memory=False)

# ✅ التأكد من وجود Clade
if 'Clade' not in df_train.columns:
    raise ValueError("❌ العمود 'Clade' غير موجود في البيانات!")

# ✅ تحويل القيم النصية إلى أرقام باستخدام Label Encoding
encoder = LabelEncoder()
df_train['Clade'] = encoder.fit_transform(df_train['Clade'])

# 🔹 تحويل كل الأعمدة النصية إلى أرقام
for col in df_train.select_dtypes(include=['object']).columns:
    if col != 'Clade':  # نتجنب تحويل العمود المستهدف مرتين
        df_train[col] = LabelEncoder().fit_transform(df_train[col].astype(str))

# ✅ المتغيرات لتخزين أفضل القيم
best_model = None
best_f1 = 0
best_auc = 0
best_iteration = None

# ✅ تكرار العملية 4 مرات
for i in range(1, 5):
    print(f"🔹 التجربة رقم {i}")

    # 🔹 اختيار 87,500 صف عشوائيًا
    df_sample = df_train.sample(n=87500, random_state=i)

    # 🔹 تقسيم البيانات 80% تدريب و 20% تحقق
    X = df_sample.drop(columns=['Clade'])  # الميزات (features)
    y = df_sample['Clade']  # التصنيفات (labels)

    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=i, stratify=y)

    # ✅ تدريب الموديل باستخدام Random Forest
    model = RandomForestClassifier(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)

    # ✅ التنبؤ على بيانات التحقق
    y_pred = model.predict(X_val)
    y_pred_proba = model.predict_proba(X_val)  # ✅ الآن نأخذ كل الاحتمالات لكل الفئات

    # ✅ حساب F1 Score و AUC-ROC
    f1 = f1_score(y_val, y_pred, average='weighted')
    auc = roc_auc_score(y_val, y_pred_proba, multi_class="ovr")  # ✅ تمرير كل الاحتمالات لكل الفئات

    print(f"📌 التجربة {i} - F1 Score: {f1:.4f}, AUC-ROC: {auc:.4f}")

    # ✅ حفظ أفضل موديل
    if auc > best_auc:
        best_model = model
        best_f1 = f1
        best_auc = auc
        best_iteration = i

# ✅ حفظ أفضل موديل
best_model_file = "/content/drive/My Drive/Colab Notebooks/best_model.pkl"
with open(best_model_file, 'wb') as f:
    pickle.dump(best_model, f)

print(f"✅ أفضل موديل كان في التجربة {best_iteration}")
print(f"✅ F1 Score: {best_f1:.4f}, AUC-ROC: {best_auc:.4f}")
print(f"✅ تم حفظ الموديل في: {best_model_file}")


🔹 التجربة رقم 1
📌 التجربة 1 - F1 Score: 0.8336, AUC-ROC: 0.9747
🔹 التجربة رقم 2
📌 التجربة 2 - F1 Score: 0.8382, AUC-ROC: 0.9761
🔹 التجربة رقم 3
📌 التجربة 3 - F1 Score: 0.8341, AUC-ROC: 0.9751
🔹 التجربة رقم 4
📌 التجربة 4 - F1 Score: 0.8360, AUC-ROC: 0.9760
✅ أفضل موديل كان في التجربة 2
✅ F1 Score: 0.8382, AUC-ROC: 0.9761
✅ تم حفظ الموديل في: /content/drive/My Drive/Colab Notebooks/best_model.pkl


In [ ]:
import pandas as pd
import numpy as np
import pickle
from sklearn.metrics import f1_score, roc_auc_score
from sklearn.preprocessing import LabelEncoder

# ✅ تحميل بيانات التحقق (Validation Set)
val_file = "/content/drive/My Drive/Colab Notebooks/val_split.csv"
df_val = pd.read_csv(val_file, low_memory=False)

# ✅ تحميل أفضل موديل
best_model_file = "/content/drive/My Drive/Colab Notebooks/best_model.pkl"
with open(best_model_file, 'rb') as f:
    best_model = pickle.load(f)

# ✅ التأكد من وجود Clade في البيانات
if 'Clade' not in df_val.columns:
    raise ValueError("❌ العمود 'Clade' غير موجود في بيانات التحقق!")

# ✅ تحويل القيم النصية إلى أرقام باستخدام نفس LabelEncoder المستخدم في التدريب
encoder = LabelEncoder()
df_val['Clade'] = encoder.fit_transform(df_val['Clade'])

# 🔹 تحويل كل الأعمدة النصية إلى أرقام بنفس الطريقة المستخدمة أثناء التدريب
for col in df_val.select_dtypes(include=['object']).columns:
    if col != 'Clade':  # ✅ استثناء العمود المستهدف
        df_val[col] = LabelEncoder().fit_transform(df_val[col].astype(str))

# ✅ تقسيم البيانات إلى الميزات (X) والتصنيفات (y)
X_val = df_val.drop(columns=['Clade'])
y_val = df_val['Clade']

# ✅ التنبؤ باستخدام أفضل موديل
y_pred = best_model.predict(X_val)
y_pred_proba = best_model.predict_proba(X_val)  # ✅ الآن نأخذ كل الاحتمالات لكل الفئات

# ✅ حساب F1 Score و AUC-ROC
f1 = f1_score(y_val, y_pred, average='weighted')
auc = roc_auc_score(y_val, y_pred_proba, multi_class="ovr")  # ✅ تمرير كل الاحتمالات لكل الفئات

# ✅ طباعة النتائج
print("📌 نتائج التحقق على 210,000 صف:")
print(f"✅ F1 Score: {f1:.4f}")
print(f"✅ AUC-ROC: {auc:.4f}")


📌 نتائج التحقق على 210,000 صف:
✅ F1 Score: 0.8057
✅ AUC-ROC: 0.9687


In [ ]:
import pandas as pd
import numpy as np
import pickle
from sklearn.preprocessing import LabelEncoder

# ✅ تحميل بيانات الاختبار (Test Set)
test_file = "/content/drive/My Drive/Colab Notebooks/test_features.csv"
df_test = pd.read_csv(test_file, low_memory=False)

# ✅ الاحتفاظ بـ ID الأصلي
if 'ID' in df_test.columns:
    test_ids = df_test['ID']
else:
    raise ValueError("❌ ملف الاختبار لا يحتوي على عمود 'ID'!")

# ✅ تحميل أفضل موديل
best_model_file = "/content/drive/My Drive/Colab Notebooks/best_model.pkl"
with open(best_model_file, 'rb') as f:
    best_model = pickle.load(f)

# ✅ التأكد من عدم وجود Clade لأنه مش موجود في بيانات الاختبار
if 'Clade' in df_test.columns:
    df_test = df_test.drop(columns=['Clade'])

# 🔹 تحويل كل الأعمدة النصية إلى أرقام بنفس الطريقة المستخدمة أثناء التدريب
for col in df_test.select_dtypes(include=['object']).columns:
    df_test[col] = LabelEncoder().fit_transform(df_test[col].astype(str))

# ✅ تنفيذ التوقعات باستخدام الموديل
y_pred = best_model.predict(df_test)

# ✅ تحويل التوقعات إلى DataFrame مع الـ ID
df_predictions = pd.DataFrame({
    "ID": test_ids,
    "Predicted_Clade": y_pred
})

# ✅ حفظ التوقعات في ملف CSV
output_file = "/content/drive/My Drive/Colab Notebooks/test_predictions.csv"
df_predictions.to_csv(output_file, index=False)

print(f"✅ تم حفظ التوقعات بنجاح في: {output_file}")


✅ تم حفظ التوقعات بنجاح في: /content/drive/My Drive/Colab Notebooks/test_predictions.csv


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import f1_score, roc_auc_score
from sklearn.preprocessing import LabelEncoder, StandardScaler
import pickle

# ✅ تحميل بيانات التدريب
train_file = "/content/drive/My Drive/Colab Notebooks/train_split.csv"
df_train = pd.read_csv(train_file, low_memory=False)

# ✅ التأكد من وجود Clade
if 'Clade' not in df_train.columns:
    raise ValueError("❌ العمود 'Clade' غير موجود في البيانات!")

# ✅ تحويل القيم النصية إلى أرقام باستخدام Label Encoding
encoder = LabelEncoder()
df_train['Clade'] = encoder.fit_transform(df_train['Clade'])

# 🔹 تحويل كل الأعمدة النصية إلى أرقام
for col in df_train.select_dtypes(include=['object']).columns:
    if col != 'Clade':
        df_train[col] = LabelEncoder().fit_transform(df_train[col].astype(str))

# ✅ المتغيرات لحفظ أفضل موديل
best_model = None
best_f1 = 0
best_auc = 0
best_iteration = None

# ✅ تجربة 4 مرات واختيار الأفضل
for i in range(1, 5):
    print(f"🔹 التجربة رقم {i}")

    # 🔹 اختيار 87,500 صف عشوائيًا
    df_sample = df_train.sample(n=87500, random_state=i)

    # 🔹 تقسيم البيانات 80% تدريب و 20% تحقق
    X = df_sample.drop(columns=['Clade'])
    y = df_sample['Clade']

    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=i, stratify=y)

    # ✅ تطبيع البيانات
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_val = scaler.transform(X_val)

    # ✅ تدريب الموديل باستخدام XGBoost
    model = XGBClassifier(n_estimators=200, max_depth=6, learning_rate=0.1, random_state=42, use_label_encoder=False, eval_metric="mlogloss")
    model.fit(X_train, y_train)

    # ✅ التنبؤ على بيانات التحقق
    y_pred = model.predict(X_val)
    y_pred_proba = model.predict_proba(X_val)

    # ✅ حساب F1 Score و AUC-ROC
    f1 = f1_score(y_val, y_pred, average='weighted')
    auc = roc_auc_score(y_val, y_pred_proba, multi_class="ovr")

    print(f"📌 التجربة {i} - F1 Score: {f1:.4f}, AUC-ROC: {auc:.4f}")

    # ✅ حفظ أفضل موديل
    if auc > best_auc:
        best_model = model
        best_f1 = f1
        best_auc = auc
        best_iteration = i
        best_scaler = scaler

# ✅ حفظ أفضل موديل و StandardScaler
best_model_file = "/content/drive/My Drive/Colab Notebooks/best_model_xgboost.pkl"
scaler_file = "/content/drive/My Drive/Colab Notebooks/best_scaler.pkl"

with open(best_model_file, 'wb') as f:
    pickle.dump(best_model, f)

with open(scaler_file, 'wb') as f:
    pickle.dump(best_scaler, f)

print(f"✅ أفضل موديل كان في التجربة {best_iteration}")
print(f"✅ F1 Score: {best_f1:.4f}, AUC-ROC: {best_auc:.4f}")
print(f"✅ تم حفظ الموديل في: {best_model_file}")
print(f"✅ تم حفظ StandardScaler في: {scaler_file}")


🔹 التجربة رقم 1


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [17:12:28] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


📌 التجربة 1 - F1 Score: 0.8523, AUC-ROC: 0.9810
🔹 التجربة رقم 2


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [17:14:18] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


📌 التجربة 2 - F1 Score: 0.8531, AUC-ROC: 0.9808
🔹 التجربة رقم 3


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [17:16:02] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


📌 التجربة 3 - F1 Score: 0.8512, AUC-ROC: 0.9802
🔹 التجربة رقم 4


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [17:17:44] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


📌 التجربة 4 - F1 Score: 0.8541, AUC-ROC: 0.9812
✅ أفضل موديل كان في التجربة 4
✅ F1 Score: 0.8541, AUC-ROC: 0.9812
✅ تم حفظ الموديل في: /content/drive/My Drive/Colab Notebooks/best_model_xgboost.pkl
✅ تم حفظ StandardScaler في: /content/drive/My Drive/Colab Notebooks/best_scaler.pkl


In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler
from xgboost import XGBClassifier
from sklearn.metrics import f1_score, roc_auc_score
from imblearn.over_sampling import SMOTE
import matplotlib.pyplot as plt

# ✅ تحميل بيانات التحقق
val_file = "/content/drive/My Drive/Colab Notebooks/val_split.csv"
df_val = pd.read_csv(val_file, low_memory=False)

# ✅ التحقق من القيم المفقودة
missing_values = df_val.isnull().sum()

# ✅ حذف أو استبدال القيم المفقودة
df_val = df_val.dropna()

# ✅ التأكد من وجود Clade في البيانات
if 'Clade' not in df_val.columns:
    raise ValueError("❌ العمود 'Clade' غير موجود في بيانات التحقق!")

# ✅ تحويل القيم النصية إلى أرقام باستخدام نفس LabelEncoder المستخدم في التدريب
encoder = LabelEncoder()
df_val['Clade'] = encoder.fit_transform(df_val['Clade'])

# 🔹 تحويل كل الأعمدة النصية إلى أرقام بنفس الطريقة المستخدمة أثناء التدريب
for col in df_val.select_dtypes(include=['object']).columns:
    if col != 'Clade':
        df_val[col] = LabelEncoder().fit_transform(df_val[col].astype(str))

# ✅ تقسيم البيانات إلى الميزات (X) والتصنيفات (y)
X_val = df_val.drop(columns=['Clade'])
y_val = df_val['Clade']

# ✅ تحليل توازن الفئات
class_distribution = y_val.value_counts()

# ✅ موازنة البيانات باستخدام SMOTE
smote = SMOTE(random_state=42)
X_val_balanced, y_val_balanced = smote.fit_resample(X_val, y_val)

# ✅ تطبيع البيانات
scaler = StandardScaler()
X_val_scaled = scaler.fit_transform(X_val_balanced)

# ✅ تحميل أفضل موديل XGBoost
best_model_file = "/content/drive/My Drive/Colab Notebooks/best_model_xgboost.pkl"
with open(best_model_file, 'rb') as f:
    best_model = pickle.load(f)

# ✅ التنبؤ باستخدام أفضل موديل XGBoost
y_pred = best_model.predict(X_val_scaled)
y_pred_proba = best_model.predict_proba(X_val_scaled)

# ✅ حساب F1 Score و AUC-ROC
f1 = f1_score(y_val_balanced, y_pred, average='weighted')
auc = roc_auc_score(y_val_balanced, y_pred_proba, multi_class="ovr")

# ✅ تحليل أهمية الميزات
feature_importance = best_model.feature_importances_
top_features_idx = np.argsort(feature_importance)[-10:]
top_features = X_val.columns[top_features_idx]
top_importances = feature_importance[top_features_idx]



f1, auc, missing_values, class_distribution

(0.8492558954724528,
 0.9802253145655241,
 ID       0
 AA       0
 AC       0
 AG       0
 AT       0
         ..
 TTA      0
 TTC      0
 TTG      0
 TTT      0
 Clade    0
 Length: 82, dtype: int64,
 Clade
 5    34170
 0    31274
 3    30483
 1    30130
 4    28825
 6    28618
 2    26500
 Name: count, dtype: int64)

In [ ]:
import pandas as pd
import numpy as np
import pickle
from sklearn.preprocessing import LabelEncoder
from xgboost import XGBClassifier

# ✅ تحميل بيانات الاختبار (Test Set)
test_file = "/content/drive/My Drive/Colab Notebooks/test_features.csv"
df_test = pd.read_csv(test_file, low_memory=False)

# ✅ الاحتفاظ بـ ID الأصلي
if 'ID' in df_test.columns:
    test_ids = df_test['ID']
else:
    raise ValueError("❌ ملف الاختبار لا يحتوي على عمود 'ID'!")

# ✅ تحميل أفضل موديل XGBoost
best_model_file = "/content/drive/My Drive/Colab Notebooks/best_model_xgboost.pkl"
with open(best_model_file, 'rb') as f:
    best_model = pickle.load(f)

# ✅ التأكد من عدم وجود Clade لأنه مش موجود في بيانات الاختبار
if 'Clade' in df_test.columns:
    df_test = df_test.drop(columns=['Clade'])

# 🔹 تحويل كل الأعمدة النصية إلى أرقام بنفس الطريقة المستخدمة أثناء التدريب
for col in df_test.select_dtypes(include=['object']).columns:
    df_test[col] = LabelEncoder().fit_transform(df_test[col].astype(str))

# ✅ تنفيذ التوقعات باستخدام الموديل
y_pred = best_model.predict(df_test)

# ✅ تحويل التوقعات إلى DataFrame مع الـ ID
df_predictions = pd.DataFrame({
    "ID": test_ids,
    "Predicted_Clade": y_pred
})

# ✅ حفظ التوقعات في ملف CSV
output_file = "/content/drive/My Drive/Colab Notebooks/test_predictions_xgboost.csv"
df_predictions.to_csv(output_file, index=False)

print(f"✅ تم حفظ التوقعات بنجاح في: {output_file}")


✅ تم حفظ التوقعات بنجاح في: /content/drive/My Drive/Colab Notebooks/test_predictions_xgboost.csv


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, roc_auc_score
from lightgbm import LGBMClassifier
from sklearn.preprocessing import LabelEncoder
import pickle

# ✅ تحميل بيانات التدريب الكاملة
train_file = "/content/drive/My Drive/Colab Notebooks/train_split.csv"
df_train = pd.read_csv(train_file, low_memory=False)

# ✅ التأكد من وجود Clade
if 'Clade' not in df_train.columns:
    raise ValueError("❌ العمود 'Clade' غير موجود في البيانات!")

# ✅ تحويل القيم النصية إلى أرقام باستخدام Label Encoding
encoder = LabelEncoder()
df_train['Clade'] = encoder.fit_transform(df_train['Clade'])

# 🔹 تحويل كل الأعمدة النصية إلى أرقام
for col in df_train.select_dtypes(include=['object']).columns:
    if col != 'Clade':  # نتجنب تحويل العمود المستهدف مرتين
        df_train[col] = LabelEncoder().fit_transform(df_train[col].astype(str))

# ✅ المتغيرات لتخزين أفضل القيم
best_model = None
best_f1 = 0
best_auc = 0
best_iteration = None

# ✅ تكرار العملية 4 مرات
for i in range(1, 5):
    print(f"🔹 التجربة رقم {i}")

    # 🔹 اختيار 87,500 صف عشوائيًا
    df_sample = df_train.sample(n=87500, random_state=i)

    # 🔹 تقسيم البيانات 80% تدريب و 20% تحقق
    X = df_sample.drop(columns=['Clade'])  # الميزات (features)
    y = df_sample['Clade']  # التصنيفات (labels)

    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=i, stratify=y)

    # ✅ تدريب الموديل باستخدام LightGBM
    model = LGBMClassifier(n_estimators=200, max_depth=6, learning_rate=0.1, random_state=42)
    model.fit(X_train, y_train)

    # ✅ التنبؤ على بيانات التحقق
    y_pred = model.predict(X_val)
    y_pred_proba = model.predict_proba(X_val)  # ✅ الآن نأخذ كل الاحتمالات لكل الفئات

    # ✅ حساب F1 Score و AUC-ROC
    f1 = f1_score(y_val, y_pred, average='weighted')
    auc = roc_auc_score(y_val, y_pred_proba, multi_class="ovr")  # ✅ تمرير كل الاحتمالات لكل الفئات

    print(f"📌 التجربة {i} - F1 Score: {f1:.4f}, AUC-ROC: {auc:.4f}")

    # ✅ حفظ أفضل موديل
    if auc > best_auc:
        best_model = model
        best_f1 = f1
        best_auc = auc
        best_iteration = i

# ✅ حفظ أفضل موديل
best_model_file = "/content/drive/My Drive/Colab Notebooks/best_model_lightgbm1.pkl"
with open(best_model_file, 'wb') as f:
    pickle.dump(best_model, f)

print(f"✅ أفضل موديل كان في التجربة {best_iteration}")
print(f"✅ F1 Score: {best_f1:.4f}, AUC-ROC: {best_auc:.4f}")
print(f"✅ تم حفظ الموديل في: {best_model_file}")


🔹 التجربة رقم 1
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.068688 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 20655
[LightGBM] [Info] Number of data points in the train set: 70000, number of used features: 81
[LightGBM] [Info] Start training from score -1.941221
[LightGBM] [Info] Start training from score -1.948413
[LightGBM] [Info] Start training from score -1.946911
[LightGBM] [Info] Start training from score -1.934970
[LightGBM] [Info] Start training from score -1.958793
[LightGBM] [Info] Start training from score -1.943513
[LightGBM] [Info] Start training from score -1.947712
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warn

In [ ]:
import pandas as pd
import numpy as np
import pickle
from sklearn.metrics import f1_score, roc_auc_score
from sklearn.preprocessing import LabelEncoder
import lightgbm as lgb

# ✅ تحميل بيانات التحقق (Validation Set)
val_file = "/content/drive/My Drive/Colab Notebooks/val_split.csv"
df_val = pd.read_csv(val_file, low_memory=False)

# ✅ تحميل أفضل موديل LightGBM
best_model_file = "/content/drive/My Drive/Colab Notebooks/best_model_lightgbm1.pkl"
with open(best_model_file, 'rb') as f:
    best_model = pickle.load(f)

# ✅ التأكد من وجود Clade في البيانات
if 'Clade' not in df_val.columns:
    raise ValueError("❌ العمود 'Clade' غير موجود في بيانات التحقق!")

# ✅ تحويل القيم النصية إلى أرقام باستخدام نفس LabelEncoder المستخدم في التدريب
encoder = LabelEncoder()
df_val['Clade'] = encoder.fit_transform(df_val['Clade'])

# 🔹 تحويل كل الأعمدة النصية إلى أرقام بنفس الطريقة المستخدمة أثناء التدريب
for col in df_val.select_dtypes(include=['object']).columns:
    if col != 'Clade':  # ✅ استثناء العمود المستهدف
        df_val[col] = LabelEncoder().fit_transform(df_val[col].astype(str))

# ✅ تقسيم البيانات إلى الميزات (X) والتصنيفات (y)
X_val = df_val.drop(columns=['Clade'])
y_val = df_val['Clade']

# ✅ التنبؤ باستخدام أفضل موديل LightGBM
y_pred = best_model.predict(X_val)
y_pred_proba = best_model.predict_proba(X_val)  # ✅ الآن نأخذ كل الاحتمالات لكل الفئات

# ✅ حساب F1 Score و AUC-ROC
f1 = f1_score(y_val, y_pred, average='weighted')
auc = roc_auc_score(y_val, y_pred_proba, multi_class="ovr")  # ✅ تمرير كل الاحتمالات لكل الفئات

# ✅ طباعة النتائج
print("📌 نتائج التحقق على بيانات الـ 210,000 صف:")
print(f"✅ F1 Score: {f1:.4f}")
print(f"✅ AUC-ROC: {auc:.4f}")


📌 نتائج التحقق على بيانات الـ 210,000 صف:
✅ F1 Score: 0.7735
✅ AUC-ROC: 0.9622


In [ ]:
import pandas as pd
import numpy as np
import pickle
from sklearn.preprocessing import LabelEncoder
import lightgbm as lgb

# ✅ تحميل بيانات الاختبار (Test Set)
test_file = "/content/drive/My Drive/Colab Notebooks/test_features.csv"
df_test = pd.read_csv(test_file, low_memory=False)

# ✅ الاحتفاظ بـ ID الأصلي
if 'ID' in df_test.columns:
    test_ids = df_test['ID']
else:
    raise ValueError("❌ ملف الاختبار لا يحتوي على عمود 'ID'!")

# ✅ تحميل أفضل موديل LightGBM
best_model_file = "/content/drive/My Drive/Colab Notebooks/best_model_lightgbm1.pkl"
with open(best_model_file, 'rb') as f:
    best_model = pickle.load(f)

# ✅ التأكد من عدم وجود Clade لأنه مش موجود في بيانات الاختبار
if 'Clade' in df_test.columns:
    df_test = df_test.drop(columns=['Clade'])

# 🔹 تحويل كل الأعمدة النصية إلى أرقام بنفس الطريقة المستخدمة أثناء التدريب
for col in df_test.select_dtypes(include=['object']).columns:
    df_test[col] = LabelEncoder().fit_transform(df_test[col].astype(str))

# ✅ تنفيذ التوقعات باستخدام الموديل
y_pred = best_model.predict(df_test)
y_pred_proba = best_model.predict_proba(df_test)  # للحصول على احتمالات الفئات

# ✅ تحويل التوقعات إلى DataFrame مع الـ ID
df_predictions = pd.DataFrame({
    "ID": test_ids,
    "Predicted_Clade": y_pred
})

# ✅ حفظ التوقعات في ملف CSV
output_file = "/content/drive/My Drive/Colab Notebooks/test_predictions_lgbm.csv"
df_predictions.to_csv(output_file, index=False)

print(f"✅ تم حفظ التوقعات بنجاح في: {output_file}")


✅ تم حفظ التوقعات بنجاح في: /content/drive/My Drive/Colab Notebooks/test_predictions_lgbm.csv
